## B. Predicting Concrete Compressive Strength Using Deep Learning: A Regression Approach with Keras (With Normalize data)

In [1]:
import pandas as pd
import numpy as np
import keras

2025-01-14 15:44:06.793197: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
filepath='data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.describe()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [5]:
# Splitting data into predictors and target
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

### Normalize Data

In [6]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [7]:
n_cols = predictors_norm.shape[1] # number of predictors

###  Import Keras Packages

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

### Build a Neural Network

In [9]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))  # Input layer
    model.add(Dense(10, activation='relu'))  # One hidden layer with 10 nodes and ReLU activation
    model.add(Dense(1))  # Output layer for regression (single output node)
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')  # Adam optimizer and MSE loss function
    return model

### Train and Test the Network

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

# build the model
baseline_model = regression_model()

# Fit the model on the training data
baseline_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 29ms/step - loss: 1585.1511 - val_loss: 1482.3821
Epoch 2/100
23/23 - 0s - 8ms/step - loss: 1569.4081 - val_loss: 1467.2712
Epoch 3/100
23/23 - 0s - 6ms/step - loss: 1553.7711 - val_loss: 1452.4030
Epoch 4/100
23/23 - 0s - 3ms/step - loss: 1538.2346 - val_loss: 1437.2542
Epoch 5/100
23/23 - 0s - 6ms/step - loss: 1522.4431 - val_loss: 1421.8376
Epoch 6/100
23/23 - 0s - 7ms/step - loss: 1506.3572 - val_loss: 1405.9138
Epoch 7/100
23/23 - 0s - 3ms/step - loss: 1489.7870 - val_loss: 1389.5411
Epoch 8/100
23/23 - 0s - 6ms/step - loss: 1472.3713 - val_loss: 1372.7739
Epoch 9/100
23/23 - 0s - 3ms/step - loss: 1454.3575 - val_loss: 1355.0715
Epoch 10/100
23/23 - 0s - 7ms/step - loss: 1435.4391 - val_loss: 1336.5229
Epoch 11/100
23/23 - 0s - 5ms/step - loss: 1415.5259 - val_loss: 1317.0271
Epoch 12/100
23/23 - 0s - 6ms/step - loss: 1394.6383 - val_loss: 1296.3480
Epoch 13/100
23/23 - 0s - 6ms/step - loss: 1372.9136 - val_loss: 1274.7081
Epoch 14/100
23/23 - 0s - 3ms/ste

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [11]:
predictions = baseline_model.predict(X_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [12]:
mse = mean_squared_error(y_test, predictions)
mean = np.mean(mse)
standard_deviation = np.std(mse)
print(mean, standard_deviation)

156.6950842599204 0.0


Create a list of 50 mean squared errors and report the mean and the standard deviation of the mean squared errors.

In [13]:
# Function to train and evaluate the model
def train_and_evaluate():
    # Split the data into training and test sets (30% test set)
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=None)
    
    # Build the model
    model = regression_model()
    
    # Train the model on the training data
    model.fit(X_train, y_train, epochs=100, verbose=0)  # Train for 100 epochs
    
    # Predict on the test set
    predictions = model.predict(X_test)
    
    # Compute the mean squared error
    mse = mean_squared_error(y_test, predictions)
    return mse

In [14]:
# Repeat the process 50 times
mse_list = [train_and_evaluate() for _ in range(50)]
mse_list
# Compute the mean and standard deviation of the MSEs
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

# Report the results
print(f"Mean of Mean Squared Errors: {mean_mse}")
print(f"Standard Deviation of Mean Squared Errors: {std_mse}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


Increasing the number of epochs from 50 to 100 led to a substantial reduction in mean MSE (approximately 54.18% improvement). This shows that additional training epochs helped the model learn more effectively.